In [1]:
from pathlib import Path
import pandas as pd 
from sqlalchemy import create_engine
import os

In [2]:
BASE_DIR = Path().cwd().parent
DATA_DIR = BASE_DIR / "datasets" / "productos_clasificados.csv"

df = pd.read_csv(DATA_DIR)
df['product_name']

0                                  Néctar de durazno 1 L
1                                Sardinas en salsa 155 g
2                                      Queso gouda 300 g
3                                      Frijol pinto 1 kg
4                                   Huevo blanco 12 pzas
                             ...                        
278                                         zapote negro
279                             tomate verde (tomatillo)
280                                     toronja (pomelo)
281                                                melon
282    contenedor termico unicel con tapa (hamburgues...
Name: product_name, Length: 283, dtype: object

In [3]:
import pandas as pd

# Creamos un patrón que dice:
# 1. (\d+(\.\d+)?) -> Busca números (enteros o decimales)
# 2. \s* -> Seguido de un espacio opcional
# 3. (kg|g|pzas|L|ml|oz) -> Seguido de una de estas unidades
# 4. $             -> Y que eso esté AL FINAL del texto

#patron = r'\s\d+(\.\d+)?\s*(kg|g|pzas|pz|L|ml|oz|)$'

# Reemplazamos ese patrón por "" (nada)
# case=False hace que le de igual si es Kg, KG o kg
# regex=True activa el modo "búsqueda de patrones"
#df['product_name'] = df['product_name'].str.replace(patron, '', case=False, regex=True)

# Limpieza final: .strip() quita los espacios sobrantes a los lados
df['product_name'] = df['product_name'].str.strip()

print(df['product_name'].tail(10))

273                                            tamarindo
274                                             espinaca
275                                  elote (maiz tierno)
276                                                mamey
277                                         granada roja
278                                         zapote negro
279                             tomate verde (tomatillo)
280                                     toronja (pomelo)
281                                                melon
282    contenedor termico unicel con tapa (hamburgues...
Name: product_name, dtype: object


In [4]:
import unicodedata


def normalize_text(texto):
    """
    Deja el texto en minúsculas, sin acentos ni caracteres raros,
    y sin espacios duplicados. Sirve para poder comparar nombres.
    """
    if pd.isna(texto):
        return ""

    # Pasar a minúsculas y quitar espacios al inicio y final
    texto = texto.lower().strip()

    # Quitar acentos
    texto = unicodedata.normalize("NFD", texto)
    texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")

    # Reemplazar algunos signos por espacio
    for rep in [",", ".", "(", ")", "-", "_"]:
        texto = texto.replace(rep, " ")

    # Quitar espacios duplicados
    texto = " ".join(texto.split())

    return texto

In [5]:
df = df[['product_name', 'category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days','shelf_life_freezer_days', 'perecedero']]
df['product_name'] = df['product_name'].apply(normalize_text)
df = df.drop_duplicates(subset=['product_name'], keep='first')

In [6]:
user = "root"
password = "root"
host = "localhost"
db_name = "pt"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db_name}")


df.to_sql(
    name='products', #Nombre de la tabla en mysql
    con=engine,
    if_exists='append', 
    index=False
)

147

In [7]:


# --- 1. CONFIGURACIÓN ---
BASE_DIR = Path.cwd().parent 
DATA_DIR = BASE_DIR / "datasets" / "ventas_normalizado.csv" 

user = "root"
password = "root"
host = "localhost"
db_name = "pt"
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db_name}")

# --- 2. CARGAR Y LIMPIAR DATAFRAME DE VENTAS ---
df_ventas = pd.read_csv(DATA_DIR)

#patron = r'\s\d+(\.\d+)?\s*(kg|g|pzas|pz|L|ml|oz|)$'
#df_ventas['product_name'] = df_ventas['product_name'].str.replace(patron, '', case=False, regex=True).str.strip()
#df_ventas['product_name'] = df_ventas['product_name'].apply(normalize_text)


# --- 3. TRAER LOS IDs DESDE MYSQL (EL MAPEO) ---
print("Descargando catálogo de productos...")
df_db_products = pd.read_sql("SELECT id, product_name FROM products", engine)

# Convertimos a diccionario: {'Durazno': 1, 'Arroz': 2, ...}
mapa_ids = dict(zip(df_db_products['product_name'], df_db_products['id']))

# Creamos la columna product_id reemplazando los nombres
df_ventas['product_id'] = df_ventas['product_name'].map(mapa_ids)

# (Opcional) Verificamos si alguno no cruzó
sin_id = df_ventas[df_ventas['product_id'].isna()]
if not sin_id.empty:
    print(f"ADVERTENCIA: {len(sin_id)} filas no encontraron su producto en la DB y se ignorarán.")
    print(sin_id['product_name'].unique())
    df_ventas = df_ventas.dropna(subset=['product_id']) # Los borramos para no romper la insert

# --- 4. PREPARAR COLUMNAS Y AGRUPAR ---
# Tu tabla pide: id (auto), sale_date, product_id, qty, unit, category_off...

# Renombrar columnas para coincidir con SQL
df_ventas = df_ventas.rename(columns={
    'fecha': 'sale_date',
    'ventas': 'qty'
})

# Asignar unidad por defecto (necesaria para la llave única compuesta)
# Si tu CSV no tiene columna unit, le ponemos 'piece' o la que uses
if 'unit' not in df_ventas.columns:
    df_ventas['unit'] = 'piece'

# --- PASO CRÍTICO: AGRUPAR ---
# Como tu tabla SQL tiene UNIQUE(sale_date, product_id, unit),
# si tienes 3 filas de "Arroz" el mismo día, fallará. Hay que sumarlas.
cols_agrupacion = ['sale_date', 'product_id', 'unit', 'category_off', 'perecedero', 'en_temporada', 'temp_inicio_mes', 'temp_fin_mes']

df_final = df_ventas.groupby(cols_agrupacion, as_index=False)['qty'].sum()

# --- 5. INSERTAR ---
print("Insertando ventas...")
df_final.to_sql(
    name='sales_daily',
    con=engine,
    if_exists='append',
    index=False
)
print("¡Ventas insertadas con éxito!")

Descargando catálogo de productos...
ADVERTENCIA: 21920 filas no encontraron su producto en la DB y se ignorarán.
['agua purificada 1.5 l' 'refresco coca-cola 2.5 l'
 'plato unicel pastelero no. 6 20 pzas'
 'plato unicel plano grande no. 9 25 pzas'
 'plato termico unicel no. 8 25 pzas' 'chile verde (jalapeno/serrano)'
 'vaso unicel termico 1 l (litro) 10 pzas' 'jitomate (tomate rojo)'
 'elote (maiz tierno)' 'tomate verde (tomatillo)' 'toronja (pomelo)'
 'contenedor termico unicel con tapa (hamburguesero) 50 pzas']
Insertando ventas...
¡Ventas insertadas con éxito!


In [8]:
#